In [12]:
 %load_ext line_profiler

In [1]:
from datetime import datetime
import json

import nltk.data
from nltk.tokenize import RegexpTokenizer
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize

import re

from tqdm import tqdm

import dask.dataframe as dd
from pyprojroot import here

import pandas as pd

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import pandas as pd
import dask.dataframe as dd
from dask.multiprocessing import get

import numpy as np
import timeit

from itertools import product
from functools import partial

import numpy as np

from multiprocessing import  Pool

In [2]:
def convert_term_to_regex(term):
    words = term.split(" ")
    regex_pattern = ""
    for term in words:
        regex_pattern += f"(?=.*\\b{term}\\b)"
    regex_pattern += ""
    return regex_pattern
assert convert_term_to_regex("incubation period") == "(?=.*\\bincubation\\b)(?=.*\\bperiod\\b)"

def find_matches_in_list(sentences, terms):
    """Determins if a term exists in a list of sentences
    """
    matches = {}
    for trm in terms:
        pattern = convert_term_to_regex(trm)
        for sent in sentences:
            if (re.search(pattern, sent)):
                matches[trm] = True
                break # if there is a pattern match go to the next term
    return(matches)

In [3]:
pdf = pd.read_csv(here("./data/db/final/kaggle/paper_text/document_parses_pmc_json.tsv"),
                 sep='\t', nrows=10)

In [4]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle') # nltk.download('punkt')

In [5]:
search_terms = [
    "incubation period",
    "infectiousness period",
    "recovery rate",
    "case fatality ratio",
    "case fatality rate",
    "asymptomatic fraction",
    "asymptomatic proportion",
    "asymptomatic ratio",
    "hospitalized fraction",
    "hospitalized proportion",
    "latent period",
]

In [7]:
bool(search_terms[0].search('period of incubation'))

AttributeError: 'str' object has no attribute 'search'

In [8]:
tqdm.pandas(desc="Split sentences")
start_time = datetime.now()

pdf["text_sent_lower"] = pdf["text"].progress_apply(lambda x: sent_detector.tokenize(x.lower().strip()))

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

C:\Users\Danie\miniconda3\envs\db_covid19\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Split sentences: 100%|█████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 57.47it/s]

Duration: 0:00:00.199001


In [9]:
start_time = datetime.now()

pdf["text_sent_lower"] = pdf["text"].apply(lambda x: sent_detector.tokenize(x.lower().strip()))

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.176000


# pandas

In [10]:
tqdm.pandas(desc="Finding terms")

start_time = datetime.now()
pdf['found_terms'] = pdf['text_sent_lower'].progress_apply(find_matches_in_list, terms=search_terms)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Finding terms: 100%|███████████████████████████████████████████████████████████████████| 10/10 [00:15<00:00,  1.53s/it]

Duration: 0:00:15.261056


In [13]:
%lprun -f find_matches_in_list pdf["found_terms"] = pdf["text_sent_lower"].progress_apply(find_matches_in_list, terms=search_terms)

Finding terms: 100%|███████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.46s/it]


Timer unit: 1e-07 s

Total time: 14.499 s
File: <ipython-input-2-5844719b7f72>
Function: find_matches_in_list at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def find_matches_in_list(sentences, terms):
    11                                               """Determins if a term exists in a list of sentences
    12                                               """
    13        10        301.0     30.1      0.0      matches = {}
    14       120       1653.0     13.8      0.0      for trm in terms:
    15       110      17787.0    161.7      0.0          pattern = convert_term_to_regex(trm)
    16     14846     284649.0     19.2      0.2          for sent in sentences:
    17     14738  144685691.0   9817.2     99.8              if (re.search(pattern, sent)):
    18         2         82.0     41.0      0.0                  matches[trm] = True
    19         2         35.0     17.5      0.0                  break 

In [14]:
pdf.head()

,pid,num_authors,title,text,text_sent_lower,found_terms
0,PMC1054884,7,Recombination Every Day: Abundant Recombinatio...,As increasing numbers of full-length viral seq...,[as increasing numbers of full-length viral se...,{}
1,PMC1065028,1,Why can't I visit? The ethics of visitation re...,The sudden emergence of severe acute respirato...,[the sudden emergence of severe acute respirat...,{}
2,PMC1065064,8,Prospective evaluation of an internet-linked h...,The rate of expansion of medical knowledge is ...,[the rate of expansion of medical knowledge is...,{}
3,PMC1065120,4,Scanning the horizon: emerging hospital-wide t...,This series of articles provides regular surve...,[this series of articles provides regular surv...,{}
4,PMC1065257,3,Characterization of the frameshift signal of E...,Programmed −1 ribosomal frameshifting (hereaft...,[programmed −1 ribosomal frameshifting (hereaf...,{}


# Speed up

In [15]:
def sent_list_to_list_set_tokens(list_of_sentences):
    return [set(word_tokenize(sent)) for sent in list_of_sentences]

In [33]:
start_time = datetime.now()

pdf["sent_set"] = pdf["text_sent_lower"].apply(sent_list_to_list_set_tokens)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:00.683998


In [35]:
%lprun -f sent_list_to_list_set_tokens pdf["sent_set"] = pdf["text_sent_lower"].apply(sent_list_to_list_set_tokens)

Timer unit: 1e-07 s

Total time: 1.03239 s
File: <ipython-input-15-7fb67494139a>
Function: sent_list_to_list_set_tokens at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents

In [36]:
pdf.loc[0, 'text_sent_lower'][0]

'as increasing numbers of full-length viral sequences become available, recombinant or mosaic viruses are being recognized more frequently [1,2,3].'

In [37]:
pdf.loc[0, 'sent_set'][0]

{',',
 '.',
 '1,2,3',
 '[',
 ']',
 'are',
 'as',
 'available',
 'become',
 'being',
 'frequently',
 'full-length',
 'increasing',
 'more',
 'mosaic',
 'numbers',
 'of',
 'or',
 'recognized',
 'recombinant',
 'sequences',
 'viral',
 'viruses'}

In [38]:
def find_set_in_list(sent_set, terms):
    """Determins if a term exists in a list of sentences
    """
    matches = {}
    for trm in terms:
        trm_set = set(trm.split(" "))
        for st in sent_set:
            if (trm_set.issubset(st)):
                matches[trm] = True
                break # if there is a pattern match go to the next term
    return(matches)

In [39]:
test_sent = ['dan hello', 'hello you', 'the quick brown fox', 'my I have a word?']
test_sent_set = [set(word_tokenize(sent)) for sent in test_sent]
test_sent_set

[{'dan', 'hello'},
 {'hello', 'you'},
 {'brown', 'fox', 'quick', 'the'},
 {'?', 'I', 'a', 'have', 'my', 'word'}]

In [40]:
find_set_in_list(test_sent_set,
                  ['hello dan', 'fox', 'tom nook', 'word'])

{'hello dan': True, 'fox': True, 'word': True}

In [41]:
%lprun -f find_set_in_list pdf["found_terms"] = pdf["sent_set"].progress_apply(find_set_in_list, terms=search_terms)

Finding terms: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 163.93it/s]


Timer unit: 1e-07 s

Total time: 0.033616 s
File: <ipython-input-38-418cf8c122eb>
Function: find_set_in_list at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def find_set_in_list(sent_set, terms):
     2                                               """Determins if a term exists in a list of sentences
     3                                               """
     4        10        180.0     18.0      0.1      matches = {}
     5       120       1206.0     10.1      0.4      for trm in terms:
     6       110       3687.0     33.5      1.1          trm_set = set(trm.split(" "))
     7     14867     130364.0      8.8     38.8          for st in sent_set:
     8     14758     200608.0     13.6     59.7              if (trm_set.issubset(st)):
     9         1         16.0     16.0      0.0                  matches[trm] = True
    10         1          9.0      9.0      0.0                  break # if there is a patte

In [29]:
find = set(['daniel', 'chen'])
find

{'chen', 'daniel'}

In [30]:
sent = set(word_tokenize('hello, my name is daniel, last name: chen.'))
sent

{',', '.', ':', 'chen', 'daniel', 'hello', 'is', 'last', 'my', 'name'}

In [31]:
sent.issubset(find)

False

In [32]:
['daniel', 'chen'] in ['hello', 'my', 'name']

False